In [1]:
import pandas as pd
from crawlers.url_crawlers import get_our_world_in_data
from filters.base_filters import keep_columns_by_name

In [2]:
raw_data = get_our_world_in_data()
filtered_data = keep_columns_by_name(raw_data, ["iso_code", "date", "new_cases", "total_cases"])

In [48]:
def fill_nans_by_interpolation(data):
    """_summary_
    Interpolation of the given data with the given attributes.
    Args:
        data (Dataframe): data to interpolate
        attributes (list): attributes for interpolating

    Returns:
        _type_: _description_
    """
    # Sort by date and then set date as index; needed for interpolation
    data = data.sort_values(by=["date"]).set_index("date")
    attributes = list(data.describe().columns)
    for attr in attributes:
        column_data = data[attr]
        if column_data.isnull().all():
            column_data.iloc[::] = 0
        else:
            if pd.isna(column_data.iloc[0]):
                column_data.iloc[0] = column_data.loc[column_data.first_valid_index()]
            if pd.isna(column_data.iloc[-1]):
                column_data.iloc[-1] = column_data.loc[column_data.last_valid_index()]
            
        column_data_resampled = column_data.resample("1d").asfreq() # Resample to daily
        column_data_interpolated = column_data_resampled.interpolate(method='linear', order=1) # Interpolation of the data
        # save to overwrite the original data
        data[attr] = column_data_interpolated
    return data.reset_index()

In [52]:
def country_based_interpolation(df):
  output = pd.DataFrame()

  unique_countries = df.iso_code.unique()
  for country in unique_countries:
    data = df[df.iso_code == country]
    # sending a slice of the data [a copy]
    interpolated_date = fill_nans_by_interpolation(data)
    output = pd.concat([output, interpolated_date])

  return output

In [53]:
out = country_based_interpolation(filtered_data)

/tmp/ipykernel_52677/3183923946.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  column_data.iloc[0] = column_data.loc[column_data.first_valid_index()]
/tmp/ipykernel_52677/3183923946.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  column_data.iloc[0] = column_data.loc[column_data.first_valid_index()]
/tmp/ipykernel_52677/3183923946.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  column_data.iloc[-1] = column_data.loc[col

In [54]:
out

,date,iso_code,new_cases,total_cases
0,2020-02-24,AFG,5.0,5.0
1,2020-02-25,AFG,0.0,5.0
2,2020-02-26,AFG,0.0,5.0
3,2020-02-27,AFG,0.0,5.0
4,2020-02-28,AFG,0.0,5.0
...,...,...,...,...
969,2022-11-14,ZWE,0.0,257893.0
970,2022-11-15,ZWE,0.0,257893.0
971,2022-11-16,ZWE,0.0,257893.0
972,2022-11-17,ZWE,0.0,257893.0
